In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
#check that optimal workforce size is the same everywhere

list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#remove berlin
val_rmv = []
for val in list_city_dt:
    if val.find('berlin') > -1:
        val_rmv.append(val)
for val in val_rmv:
    list_city_dt.remove(val)

list_city_dt

['frankfurt_db=0.50',
 'frankfurt_db=1.00',
 'frankfurt_db=2.00',
 'frankfurt_db=4.00',
 'lyon_db=0.50',
 'lyon_db=1.00',
 'lyon_db=2.00',
 'lyon_db=4.00',
 'paris_db=0.50',
 'paris_db=1.00',
 'paris_db=2.00',
 'paris_db=4.00']

In [3]:
list_curr = []

for base_file in list_city_dt:
    workforce_file = f'../../workforce_size/{base_file}.json'
    with open(workforce_file, 'r') as file:
        data = json.load(file)
        df_workforce = pd.DataFrame(data)
        list_curr.append(df_workforce)

list_rerun = []
for base_file in list_city_dt:
    workforce_file = f'../../rerun_workforce_size/{base_file}.json'
    with open(workforce_file, 'r') as file:
        data = json.load(file)
        df_workforce = pd.DataFrame(data)
        list_rerun.append(df_workforce)

In [4]:
df_curr = pd.concat(list_curr, ignore_index = True)
df_rerun = pd.concat(list_rerun, ignore_index = True)

df_curr.loc[df_curr['max_n_shifts'].isna(), 'max_n_shifts'] = 0
df_rerun.loc[df_rerun['max_n_shifts'].isna(), 'max_n_shifts'] = 0

list_keep = ['instance_file_base','model',
       'max_n_shifts', 'outsourcing_cost_multiplier','workforce_size_region0','objective_value']
list_rename = ['instance','model','max_n_shifts','OC','workforce_size','objval']

df_curr = df_curr[list_keep]
df_curr.columns = list_rename
df_rerun = df_rerun[list_keep]
df_rerun.columns = list_rename

print(df_curr.columns)
print(df_rerun.columns)

Index(['instance', 'model', 'max_n_shifts', 'OC', 'workforce_size', 'objval'], dtype='object')
Index(['instance', 'model', 'max_n_shifts', 'OC', 'workforce_size', 'objval'], dtype='object')


In [7]:
df_compare = df_curr.merge(df_rerun, how = 'left', on = ['instance','model','max_n_shifts','OC'])

df_compare.columns = ['instance', 'model', 'max_n_shifts', 'OC', 'workforce_size_curr',
       'objval_curr', 'workforce_size_rerun', 'objval_rerun']

df_compare.sort_values(by = ['instance','model','max_n_shifts', 'OC'], inplace = True)
df_view = df_compare[df_compare['workforce_size_curr']!=df_compare['workforce_size_rerun']]
print(len(df_view['instance']))

df_compare.to_excel(r'../code_cm/compare_workforce_sizes.xlsx', index = False)
df_view


97


,instance,model,max_n_shifts,OC,workforce_size_curr,objval_curr,workforce_size_rerun,objval_rerun
2,frankfurt_db=0.50,partflex,2.0,1.2,15,1768.080000,13,1768.08
7,frankfurt_db=0.50,partflex,2.0,1.5,17,2125.300000,15,2125.30
12,frankfurt_db=0.50,partflex,2.0,1.8,20,2479.200000,17,2479.20
35,frankfurt_db=1.00,fixed,0.0,1.8,23,5235.740000,22,5235.74
40,frankfurt_db=1.00,fixed,0.0,2.0,24,5753.488889,22,5753.49
...,...,...,...,...,...,...,...,...
298,paris_db=4.00,partflex,3.0,2.5,191,121566.913802,192,121565.69
279,paris_db=4.00,partflex,4.0,1.2,178,60685.730665,180,60685.46
284,paris_db=4.00,partflex,4.0,1.5,183,74760.069922,182,74760.85
289,paris_db=4.00,partflex,4.0,1.8,184,88819.870140,185,88819.58
